In [54]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off

from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.20 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.


In [3]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [4]:
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

In [5]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [6]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)

In [7]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible,
while being safe. Your answers should not include any harmful, unethical, racist,
sexist, toxic, dangerous, or illegal content. Please ensure that your responses are
socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead
of answering something not correct. If you don't know the answer to a question, please
don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
text_pipeline = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
  max_new_tokens=1024,
  temperature=0,
  top_p=0.95,
  repetition_penalty=1.15,
  streamer=streamer,
)
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
  """
{context}

Question: {question}
""",
  system_prompt=SYSTEM_PROMPT,
)
prompt = PromptTemplate(template=template, input_variables=["context", "question"])


    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.1.0+cu121)
    Python  3.10.11 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'Marian

In [8]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [9]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [27]:
pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.6 MB/s eta 0:00:00


In [10]:
from langchain.document_loaders import GutenbergLoader

In [11]:
!pip install flask-ngrok
!pip install pyngrok


In [30]:
!ngrok authtoken '2XUaR4kfgbvxOhylu6vq9kYmxsB_5hp5PhZSuR6AVQy8J5ATH'
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [13]:

!pip install flask

In [14]:
!pip install streamlit

In [15]:
!pip install flask-ngrok

In [16]:
pip install transformers

In [17]:
from transformers import AutoTokenizer

# Define the model and tokenizer names
model_name = "pszemraj/led-large-book-summary"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [18]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [42]:
from transformers import pipeline

summarizer = pipeline("summarization", model="pszemraj/led-large-book-summary", device=0 if torch.cuda.is_available() else -1)

In [20]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import streamlit as st
import requests
from bs4 import BeautifulSoup
import difflib

In [50]:
import requests
from bs4 import BeautifulSoup
import difflib

# Function to search for a book by name and return the best match URL
def search_book_by_name(book_name):
    base_url = "https://www.gutenberg.org/"
    search_url = base_url + "ebooks/search/?query=" + book_name.replace(" ", "+") + "&submit_search=Go%21"

    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the best match link based on similarity ratio
    best_match_ratio = 0
    best_match_url = ""

    for link in soup.find_all("li", class_="booklink"):
        link_title = link.find("span", class_="title").get_text()
        similarity_ratio = difflib.SequenceMatcher(None, book_name.lower(), link_title.lower()).ratio()
        if similarity_ratio > best_match_ratio:
            best_match_ratio = similarity_ratio
            best_match_url = base_url + link.find("a").get("href")

    return best_match_url

# Function to get the "Plain Text UTF-8" download link from the book page
def get_plain_text_link(book_url):
    response = requests.get(book_url)
    soup = BeautifulSoup(response.content, "html.parser")

    plain_text_link = ""

    for row in soup.find_all("tr"):
        format_cell = row.find("td", class_="unpadded icon_save")
        if format_cell and "Plain Text UTF-8" in format_cell.get_text():
            plain_text_link = format_cell.find("a").get("href")
            break

    return plain_text_link


# Function to get the content of the "Plain Text UTF-8" link
def get_plain_text_content(plain_text_link):
    response = requests.get(plain_text_link)
    content = response.text
    return content
def get_sentences(book_name):
    print(book_name)
    best_match_url = search_book_by_name(book_name)
    if best_match_url:
        plain_text_link = get_plain_text_link(best_match_url)
        if plain_text_link:
            full_plain_text_link = "https://www.gutenberg.org" + plain_text_link
            FileContent = get_plain_text_content(full_plain_text_link)
            # unstring "plain_text_content" to print the whole book
            FileContent = FileContent.split("*** START OF THE PROJECT GUTENBERG EBOOK")[1]
            FileContent = FileContent.split("*** END OF THE PROJECT GUTENBERG EBOOK THE PRINCE")[0]
            # extract the sentences from the document
            sentences = nltk.tokenize.sent_tokenize(FileContent)
            return sentences
        else:
            print("No Plain Text UTF-8 link found.")
    else:
        print("No matching book found.")
def get_chunks(sentences):
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1

        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
        if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
            chunk += sentence + " " # add the sentence to the chunk
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
              chunks.append(chunk.strip()) # save the chunk

        else:
            chunks.append(chunk.strip()) # save the chunk

          # reset
            length = 0
            chunk = ""

            # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))
    print(chunks)
    return chunks

def generate_summary(chunks):
    i=0
    print("Hi")
    for inputs in chunks:
        print("running")
        i+=1
        summary = summarizer(inputs, max_length=150, min_length=30,
                          no_repeat_ngram_size=3,
                          encoder_no_repeat_ngram_size=3,
                          repetition_penalty=3.5,
                          num_beams=4,
                          do_sample=False)
        final_summary = summary[0]['summary_text']
        print('hi')
        return final_summary

# Tested book names
#     -The changed brides
#     -The bride's fate

In [22]:
pip install pyngrok

In [23]:
from flask import Flask

In [24]:
import requests
from bs4 import BeautifulSoup
import difflib

# Function to search for a book by name and return the best match URL
def search_book(book_name):
    base_url = "https://www.gutenberg.org/"
    search_url = base_url + "ebooks/search/?query=" + book_name.replace(" ", "+") + "&submit_search=Go%21"

    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the best match link based on similarity ratio
    best_match_ratio = 0
    best_match_url = ""

    for link in soup.find_all("li", class_="booklink"):
        link_title = link.find("span", class_="title").get_text()
        similarity_ratio = difflib.SequenceMatcher(None, book_name.lower(), link_title.lower()).ratio()
        if similarity_ratio > best_match_ratio:
            best_match_ratio = similarity_ratio
            best_match_url = base_url + link.find("a").get("href")

    return best_match_url

def get_loader(book_name):
    best_match_url = search_book(book_name)

    if best_match_url:
        url_parts = best_match_url.rsplit('/', 1)
        booknum = url_parts[1]
        exact_url = "https://www.gutenberg.org/cache/epub/"+booknum+"/pg"+booknum+".txt"
        loader = GutenbergLoader(exact_url)
        return loader

    else:
        print("No matching book found.")



# Tested book names
#     -Jane Eyre: An Autobiography
#     -The changed brides
#     -The bride's fate


In [37]:
def QA(loader,query):
    data = loader.load()
    !rm -rf "db"
    texts = text_splitter.split_documents(data)
    print("Hi",texts)
    db = Chroma.from_documents(texts, embeddings, persist_directory="db")
    llm_ans = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type="stuff",
      retriever=db.as_retriever(search_kwargs={"k": 2}),
      return_source_documents=True,
      chain_type_kwargs={"prompt": prompt},
    )
    result = llm_ans(query)
    return result

In [55]:
# Import your summarization and Q/A models here
# Replace these with your actual model import code
app = Flask(__name__)
run_with_ngrok(app)
@app.route('/')
def home():
    return "Homepage of Book Summarization"
@app.route('/get_summary', methods=['POST'])
def get_summary():
    try:
        data = request.get_json()
        book_name = data['book_name']
        sentences = get_sentences(book_name)
        chunks = get_chunks(sentences)
        summary = generate_summary(chunks)
        return jsonify({"summary": summary})
    except Exception as e:
        return jsonify({'error': str(e)}), 500
@app.route('/get_answer', methods=['POST'])
def get_answer():
    try:
        data = request.get_json()
        book_name = data['book_name']
        question = data['question']
        loader = get_loader(book_name)
        answer = QA(loader, question)
        answer = answer['result']
        return jsonify({"answer": answer})
    except Exception as e:
        return jsonify({'error': str(e)}), 500
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://36c6-34-125-201-121.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
The prince


INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 18:40:21] "POST /get_summary HTTP/1.1" 500 -


['THE PRINCE ***\r\n\r\n\r\n\r\nThe Prince\r\n\r\nby Nicolo Machiavelli\r\n\r\nTranslated by W. K. Marriott\r\n\r\n\r\nContents\r\n\r\n INTRODUCTION\r\n YOUTH Æt. 1-25—1469-94\r\n OFFICE Æt. 25-43—1494-1512\r\n LITERATURE AND DEATH Æt. 43-58—1512-27\r\n THE MAN AND HIS WORKS\r\n DEDICATION\r\n\r\n THE PRINCE\r\n CHAPTER I. HOW MANY KINDS OF PRINCIPALITIES THERE ARE, AND BY WHAT MEANS THEY ARE ACQUIRED\r\n CHAPTER II. CONCERNING HEREDITARY PRINCIPALITIES\r\n CHAPTER III. CONCERNING MIXED PRINCIPALITIES\r\n CHAPTER IV. WHY THE KINGDOM OF DARIUS, CONQUERED BY ALEXANDER, DID NOT REBEL AGAINST THE SUCCESSORS OF ALEXANDER AT HIS DEATH\r\n CHAPTER V. CONCERNING THE WAY TO GOVERN CITIES OR PRINCIPALITIES WHICH LIVED UNDER THEIR OWN LAWS BEFORE THEY WERE ANNEXED\r\n CHAPTER VI. CONCERNING NEW PRINCIPALITIES WHICH ARE ACQUIRED BY ONE’S OWN ARMS AND ABILITY\r\n CHAPTER VII. CONCERNING NEW PRINCIPALITIES WHICH ARE ACQUIRED EITHER BY THE ARMS OF OTHERS OR BY GOOD FORTUNE\r\n CHAPTER VIII. CONCERNIN

INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 19:07:48] "POST /get_answer HTTP/1.1" 200 -
